<a href="https://colab.research.google.com/github/JU100/obsidian-css-snippets/blob/master/Clean_up_Google_Play_Highlights_(To_use_go_to_File_%3E_Save_a_Copy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Google Drive to access highlight documents

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Authenticate Google Drive API

In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

## Function to clean up highlights 

In [ ]:
import io
from googleapiclient.http import MediaIoBaseDownload
from google.colab import files


def clean_highlights(file_id):
  request = drive_service.files().export(fileId=file_id, mimeType="text/plain") # request Gdoc as plain text
  downloaded = io.BytesIO() 
  downloader = MediaIoBaseDownload(downloaded, request) # get requested file as bytestream
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = downloader.next_chunk()
  downloaded.seek(0) # go to first position in bytestream
  x = downloaded.read().decode("utf-8") # decode bytestream -> convert to str
  l = x.split('\t') # split str into list and only take text fields (that start with \t) NOTE: This omits the automatically added chapter titles in the gdoc -> user has to highlight chapters if wants them
  l1 = l[4:]  # strip stuff in the beginning
  l2 = l1[::4] # only take actual text blocks, not page number and empty lines
  l3 = [] 
  for line in l2:
    l3.append(line.split('\r\n\r\n\r\n')[0]) # strip date when highlight was taken from text block
  with open("output.txt", "w") as output: # write to new txt file 
     for line in l3:
        output.write("%s\n\n" % line) # add new lines between text blocks
  files.download("output.txt") # download file


## Run function with your file ID
The file ID is the block in the Google Doc URL after https://docs.google.com/document/d/

In [ ]:
clean_highlights(file_id = '18lw432_T9hCi2FeegQICW2HwAZD0-5WiQw9GqyzUwpg')

HttpError: ignored

## Flush and unmount

In [ ]:
drive.flush_and_unmount()